<a href="https://colab.research.google.com/github/FelosRG/MNIST-Android/blob/main/MNIST_Android.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST KERAS

In [1]:
# REVISAMOS EL DISPOSITIVO EN EL QUE ESTAMOS TRABAJANDO

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2726610591092874326
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10041975974662050738
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3570962965847598056
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14640891840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12700630950342469278
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [2]:
# SETUP

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf


In [3]:
# PREPARACIÓN DE DATOS

num_classes = 10
input_shape = (28, 28, 1)

# Separamos entre los conjuntos de train y test
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalización
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Convertimos los tesores a la forma adecuada.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# Pasamos los labels a one-hot econding 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# MODELO

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [5]:
# ENTRENAMIENTO

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 2s 5ms/step - loss: 0.3643 - accuracy: 0.8892 - val_loss: 0.0829 - val_accuracy: 0.9785
Epoch 2/15
422/422 [==============================] - 2s 5ms/step - loss: 0.1144 - accuracy: 0.9656 - val_loss: 0.0626 - val_accuracy: 0.9828
Epoch 3/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0865 - accuracy: 0.9737 - val_loss: 0.0466 - val_accuracy: 0.9858
Epoch 4/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0725 - accuracy: 0.9775 - val_loss: 0.0422 - val_accuracy: 0.9878
Epoch 5/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0632 - accuracy: 0.9804 - val_loss: 0.0376 - val_accuracy: 0.9902
Epoch 6/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0556 - accuracy: 0.9827 - val_loss: 0.0363 - val_accuracy: 0.9898
Epoch 7/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0520 - accuracy: 0.9840 - val_loss: 0.0365 - val_accuracy: 0.9905
Epoch 

In [6]:
# EVALUACIÓN DEL MODELO

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.027707671746611595
Test accuracy: 0.9901999831199646


In [10]:
# Guardanos el modelo de tensorlfow
name = "/home/MNIST_andoid_model.HDF5"
model.save(name)

INFO:tensorflow:Assets written to: /home/MNIST_andoid_model.HDF5/assets


INFO:tensorflow:Assets written to: /home/MNIST_andoid_model.HDF5/assets


In [11]:
# Convert the model

converter = tf.lite.TFLiteConverter.from_saved_model(name) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)